# My proposal for Titaniv Challenge

#### For this notebook I will inspire my self of the EDA we have done in class and will try to implement some more to get a more 

#### accurate result without overfitting the data

### Import packages 

In [251]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bays
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

### Load the data

In [252]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
combine = [train_df, test_df]

### Data exploration

We won't do all the exploration as we have already done a lot in class

In [253]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


As seen before there are a lot of categorical data that we will need to process

Let's drop the columns we have no clue how to use

In [254]:
train_df = train_df.drop(['PassengerId','Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

Look for correlations. We will look again after changing categorical values

In [255]:
corr_matrix = train_df.corr()
corr_matrix["Survived"].sort_values(ascending=False)

Survived    1.000000
Fare        0.257307
Parch       0.081629
SibSp      -0.035322
Age        -0.077221
Pclass     -0.338481
Name: Survived, dtype: float64

In [256]:
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset = dataset.drop("Name", axis=1, inplace=True)

In [257]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


Let's try something different to fill the afe NaN Values

In [258]:
Titles = ["Mr", "Mrs", "Miss", "Rare", "Master"]

for dataset in combine:
    for title in Titles:
        mean_age = dataset.dropna().loc[dataset.Title == title, "Age"].mean()
        dataset.loc[(dataset.Age.isnull())&(dataset.Title == title), "Age"] = mean_age

    


In [259]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,892,3,male,34.5,0,0,7.8292,Q,Mr
1,893,3,female,47.0,1,0,7.0000,S,Mrs
2,894,2,male,62.0,0,0,9.6875,Q,Mr
3,895,3,male,27.0,0,0,8.6625,S,Mr
4,896,3,female,22.0,1,1,12.2875,S,Mrs


Use the dummmies technic

In [260]:
for dataset in combine:
    dummies = pd.get_dummies(dataset['Title'])
    columns = dummies.columns
    dataset[columns] = dummies
    dataset = dataset.drop(["Title"], axis=1, inplace=True)

In [261]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Master,Miss,Mr,Mrs,Rare
0,0,3,male,22.0,1,0,7.2500,S,0,0,1,0,0
1,1,1,female,38.0,1,0,71.2833,C,0,0,0,1,0
2,1,3,female,26.0,0,0,7.9250,S,0,1,0,0,0
3,1,1,female,35.0,1,0,53.1000,S,0,0,0,1,0
4,0,3,male,35.0,0,0,8.0500,S,0,0,1,0,0


Same for Sex

In [262]:
for dataset in combine:
    dataset["Sex"] = dataset["Sex"].replace("male", 0)
    dataset["Sex"] = dataset["Sex"].replace("female", 1)

In [263]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Master,Miss,Mr,Mrs,Rare
0,0,3,0,22.0,1,0,7.2500,S,0,0,1,0,0
1,1,1,1,38.0,1,0,71.2833,C,0,0,0,1,0
2,1,3,1,26.0,0,0,7.9250,S,0,1,0,0,0
3,1,1,1,35.0,1,0,53.1000,S,0,0,0,1,0
4,0,3,0,35.0,0,0,8.0500,S,0,0,1,0,0


In [264]:
train_df.Age.isnull().sum()

0

We have categorical data for embarked column. Once again let's use the dummies technique

In [265]:
freq_port = train_df['Embarked'].dropna().mode()[0]
for dataset in combine:
       dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)


In [266]:
train_df.Embarked.isnull().sum()

0

In [267]:
for dataset in combine:
    dummies = pd.get_dummies(dataset['Embarked'])
    columns = dummies.columns
    dataset[columns] = dummies
    dataset = dataset.drop("Embarked", axis=1, inplace=True)
    
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Master,Miss,Mr,Mrs,Rare,C,Q,S
0,0,3,0,22.0,1,0,7.2500,0,0,1,0,0,0,0,1
1,1,1,1,38.0,1,0,71.2833,0,0,0,1,0,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,1,0,0,0,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,0,1,0,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1,0,0,0,0,1


In [268]:
for dataset in combine:
    dataset["Family"] = dataset["SibSp"] + dataset["Parch"]
    dataset["Family"].loc[dataset['Family'] > 0] = 1
    dataset["Family"].loc[dataset['Family'] == 0] = 0 
    dataset.drop(["SibSp","Parch"], axis=1, inplace=True)
    
train_df.head()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Survived,Pclass,Sex,Age,Fare,Master,Miss,Mr,Mrs,Rare,C,Q,S,Family
0,0,3,0,22.0,7.2500,0,0,1,0,0,0,0,1,1
1,1,1,1,38.0,71.2833,0,0,0,1,0,1,0,0,1
2,1,3,1,26.0,7.9250,0,1,0,0,0,0,0,1,0
3,1,1,1,35.0,53.1000,0,0,0,1,0,0,0,1,1
4,0,3,0,35.0,8.0500,0,0,1,0,0,0,0,1,0


In [269]:
test_df["Fare"].fillna(test_df["Fare"].median(), inplace=True)

In [270]:
test_df.Fare.isnull().sum()

0

In [271]:
train_df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Master      0
Miss        0
Mr          0
Mrs         0
Rare        0
C           0
Q           0
S           0
Family      0
dtype: int64

In [272]:
for dataset in combine:
    dataset["Fare"] = dataset["Fare"].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Master,Miss,Mr,Mrs,Rare,C,Q,S,Family
0,0,3,0,22.0,7,0,0,1,0,0,0,0,1,1
1,1,1,1,38.0,71,0,0,0,1,0,1,0,0,1
2,1,3,1,26.0,7,0,1,0,0,0,0,0,1,0
3,1,1,1,35.0,53,0,0,0,1,0,0,0,1,1
4,0,3,0,35.0,8,0,0,1,0,0,0,0,1,0


## Model training

### Decision Tree

In [273]:
X = train_df.drop("Survived", axis=1) # Training & Validation data
Y = train_df["Survived"]              # Response / Target Variable

X_submission  = test_df.drop("PassengerId", axis=1).copy()

print(X.shape, Y.shape)


(891, 13) (891,)


In [274]:
# Split training set so that we validate on 20% of the data
# Note that our algorithms will never have seen the validation 
# data during training. This is to evaluate how good our estimators are.

np.random.seed(1337) # set random seed for reproducibility

from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

(712, 13) (712,)
(179, 13) (179,)


In [275]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
# Y_pred = decision_tree.predict(X_test)
print(round(decision_tree.score(X_train, Y_train) * 100, 2))


96.91


In [276]:
missing = X_submission.isnull().sum().sort_values(ascending = False)
print(missing)


Family    0
S         0
Q         0
C         0
Rare      0
Mrs       0
Mr        0
Miss      0
Master    0
Fare      0
Age       0
Sex       0
Pclass    0
dtype: int64


In [277]:
Y_submission = decision_tree.predict(X_submission)
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_submission
    })
submission.to_csv('titanic.csv', index=False)